In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from task1 import create_csv1
import re
import string
import pymorphy2
morph = pymorphy2.MorphAnalyzer()


In [3]:
def create_columns(name_class_good: str, name_class_bad: str, df: pd.DataFrame) -> None:
    '''create columns of DataFrame'''
    df['review'] = ""
    df['number_words'] = ""
    i = 0
    for file_dataset in os.listdir("dataset"):
        path_p = os.path.join("dataset", file_dataset)
        class_name = os.listdir(path_p)
        for elem in class_name:
            name = os.path.join(path_p, elem)
            f = open(name, mode='rt', encoding='utf-8')
            data = f.read()
            df.loc[[i], 'review'] = data
            words = data.split()
            df.loc[[i], 'number_words'] = len(words)
            f.close()
            i += 1


def create_new_df(df: pd.DataFrame, limit: int) -> pd.DataFrame:
    '''create new DataFrame with limiting value'''
    return df[df['number_words'] <= limit][['class_name', 'review', 'number_words']]


def create_df_metka(df: pd.DataFrame, metka: str) -> pd.DataFrame:
    '''create new DataFrame by label'''
    return df[df['class_name'] == metka][['class_name', 'review', 'number_words']]


def group_df(df: pd.DataFrame) -> pd.DataFrame:
    '''grouping the data frame by label and displaying the average value, maximum and minimum of the "number_words" '''
    print("average: ", df.number_words.mean(axis=0))
    print("maximum: ", df.number_words.max())
    print("minimum", df.number_words.min())
    return df.groupby('class_name').count()


def lemmatize(text) -> list:
    '''lemmatization of the text'''
    new_list = []
    tt = str.maketrans(dict.fromkeys(string.punctuation))
    elem = text.translate(tt)
    if elem is not None:
        list_words = elem.split()
    for word in list_words:
        p = morph.parse(word)[0]
        new_list.append(p.normal_form)
    return new_list


In [52]:
def create_histog(df:pd.DataFrame, metka):
    dict = {}
    list_word = []
    list_count = []
    for text in df['review']:
        tt = str.maketrans(dict.fromkeys(string.punctuation))
        elem = text.translate(tt)
        if elem is not None:
            list_words = elem.split()
        for word in list_words:
            p = morph.parse(word)[0]
            list_word.append(p.normal_form)
    for word in list_word:
        if word not in dict.keys():
            dict[word] = list_word.count(word)
    fig = plt.figure(figsize=(15, 10))
    plt.ylabel('number of words')
    plt.xlabel('words')
    plt.hist(dict.keys(), bins=2, color='salmon') 
    return plt


create DataFrame

In [43]:
name_class_good = "good"
name_class_bad = "bad"
create_csv1(name_class_bad, name_class_good, "annotation")
df = pd.read_csv("annotation", encoding='utf-16', delimiter=';',
                 names=['absolut_path', 'otnos_path', 'class_name'])
df = df.drop(['absolut_path', 'otnos_path'], axis=1)
df


,class_name
0,good
1,good
2,good
3,good
4,good
...,...
1995,bad
1996,bad
1997,bad
1998,bad


checking for emptiness

In [ ]:
pd.isnull(df)

counting number of empty elements

In [ ]:
df.isnull().sum()


In [44]:
create_columns(name_class_good, name_class_bad, df)
df


,class_name,review,number_words
0,good,"Сумерки\nНа фильм попала случайно, никогда осо...",238
1,good,Сумерки\nСкажу сразу: смотрела фильм скаченный...,194
2,good,Сумерки\nК своему счастью с книгой я познакоми...,753
3,good,"Сумерки\nПризнаюсь, фильм решила посмотреть из...",281
4,good,Сумерки\nФильм 'Сумерки' реально мёртвое кино ...,332
...,...,...,...
1995,bad,"Тайна Коко\nНаверное, это будет мой самый коро...",392
1996,bad,"Тайна Коко\nБез сомнения, у студии Пиксар есть...",822
1997,bad,Тайна Коко\n'Тайна Коко' - это история о мекси...,272
1998,bad,"Тайна Коко\nК сожалению, в последнее время, та...",218


In [216]:
df_lim = create_new_df(df, 100)
df_lim


,class_name,review,number_words
1958,good,Список Шиндлера\nНасколько реально снять фильм...,82
1983,good,Список Шиндлера\nЯ редко помню о чем было прос...,86
1984,good,10 причин моей ненависти\nЭто хоть и стареньки...,35


statistical information for number_words column

In [215]:
df.number_words.describe()


count     2000
unique     132
top        281
freq        92
Name: number_words, dtype: int64

In [54]:
df_met = create_df_metka(df, name_class_good)
df_met


,class_name,review,number_words
0,good,"Сумерки\nНа фильм попала случайно, никогда осо...",238
1,good,Сумерки\nСкажу сразу: смотрела фильм скаченный...,194
2,good,Сумерки\nК своему счастью с книгой я познакоми...,753
3,good,"Сумерки\nПризнаюсь, фильм решила посмотреть из...",281
4,good,Сумерки\nФильм 'Сумерки' реально мёртвое кино ...,332
...,...,...,...
995,good,Звёздные войны: Пробуждение силы\nВдохновившис...,413
996,good,Звёздные войны: Пробуждение силы\n2005 год. Бе...,491
997,good,Звёздные войны: Пробуждение силы\nКак фанат са...,447
998,good,Звёздные войны: Пробуждение силы\nЗвездные Вой...,567


In [209]:
df_group = group_df(df)
df_group


average:  369.6215
maximum:  1791
minimum 35


,review,number_words
class_name,,
bad,1000,1000
good,1000,1000


In [53]:
create_histog(df, 'class_name')


{'сумерки': 825, 'на': 9844, 'фильм': 13287, 'попасть': 346, 'случайно': 57, 'никогда': 630, 'особо': 136, 'не': 18105, 'любить': 669, 'такой': 3659, 'род': 192, 'но': 6290, 'так': 3126, 'получиться': 595, 'что': 13557, 'прийтись': 93, 'посмотреть': 958, 'первый': 1301, 'минута': 536, '20': 156, 'я': 8720, 'всё': 4583, 'ждать': 342, 'когда': 1227, 'же': 2984, 'начаться': 139, 'главный': 1324, 'интрига': 137, 'наконец': 89, 'закрутиться': 43, 'завертеться': 43, 'судить': 230, 'по': 2639, 'тот': 4109, 'прочитать': 319, 'в': 19596, 'анонс': 43, 'почемуть': 139, 'то': 2914, 'чтобы': 1015, 'какоготь': 45, 'экшен': 160, 'хотя': 966, 'бы': 2290, 'чеготь': 208, 'захватывать': 286, 'дождаться': 52, 'касаться': 217, 'актёрский': 278, 'состав': 56, 'никто': 356, 'интересный': 478, 'ли': 770, 'талантливый': 226, 'увидеть': 874, 'обычный': 281, 'мальчикидевочка': 43, 'героиня': 332, 'свой': 4099, 'игра': 1437, 'совсем': 394, 'понравиться': 716, 'троечка': 43, 'про': 917, 'сюжет': 1519, 'написать': 